In [51]:
# モジュール
import os
import re
import tqdm
import argparse
import time
from io import StringIO
from glob import glob
import pandas as pd

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter, PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBoxHorizontal
from pdfminer.pdfpage import PDFPage

from googletrans import Translator
translator = Translator()

In [52]:
# ディレクトリ
pdf_dir = ""
path = os.path.join(pdf_dir, '*.pdf')
files = glob(path)
print(files,len(files))

['arxiv.pdf', 'arxiv2.pdf'] 2


In [75]:
#  文字数を区切ってリストに保存
def split_en_txt(str, num):
    list_en_txt = []
    for i in range(num):
        list_en_txt.append(str[i::num])
    list_en_txt = ["".join(i) for i in zip(*list_en_txt)]
    rem = len(str) % num
    if rem:
        list_en_txt.append(str[-rem:])
    return list_en_txt

#  英語から日本語に変換
def en_convert_to_ja(txt):
    ja_txt = []
    for en_txt in txt:
        trans_de_ja = translator.translate(en_txt, src='en', dest='ja')
        ja_txt.append(trans_de_ja.text)
    return ja_txt

#  分割した日本語のリストを結合
def map_list(list_txt):
    map_list = []
    ja_txt = ''
    map_list = map(str, list_txt)
    ja_txt = ''.join(map_list)
    return ja_txt

# あらかじめpandasでデータフレームを作成し、その中にテキストを入れていきます。
df = pd.DataFrame({'file_id': ['A1'],
                   'en_txt': ['B1'],
                   'ja_txt': ['C1']})
for i,path_pdf in enumerate(tqdm.tqdm(files)):
    file_id_with_version = os.path.splitext(os.path.basename(path_pdf))[0]
    
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    laparams.detect_vertical = True
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path_pdf, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    en_txt = retstr.getvalue()
    list_split_en_txt = split_en_txt(en_txt,1000)
    #  一気に日本語変換するとエラーがよくでるので60秒ほど間をおきます
    time.sleep(60)
    list_ja = en_convert_to_ja(list_split_en_txt)
    ja_txt = map_list(list_ja)
    df.loc[i,'file_id'] = file_id_with_version
    df.loc[i,'en_txt'] = en_txt
    df.loc[i,'ja_txt'] = ja_txt
    fp.close()
    device.close()
    retstr.close()  

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

74


 50%|█████████████████████████████████████████▌                                         | 1/2 [01:40<01:40, 100.17s/it]

74
35


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:54<00:00, 87.33s/it]

35


In [78]:
len(df['ja_txt'][0])

39801

In [79]:
len(df['ja_txt'][1])

18809

In [80]:
df.to_csv("test.csv",index=False,encoding='utf-8')
df.to_csv("test.tsv",sep='\t',index=False,encoding='utf-8')

In [81]:
gf = pd.read_csv("test.csv",encoding='utf-8')
gf2 = pd.read_csv("test.tsv",sep='\t',encoding='utf-8')

In [82]:
gf = pd.read_csv("test.csv",encoding='utf-8')
gf

,file_id,en_txt,ja_txt
0,arxiv,Article\nOff-Axis Characterisation of the CERN...,論文\nCERNT10ビームの軸外特性評価\n高い運動量陽子測定\n圧力ガスタイムプロジェク...
1,arxiv2,Neural Architecture Search in\nGraph Neural Ne...,ニューラルアーキテクチャ検索\nグラフニューラルネットワーク\n\nマテウス・ヌネスとジゼル...


In [65]:
gf["ja_txt"][0]

'論文\nCERNT10ビームの軸外特性評価\n高い運動量陽子測定\n圧力ガスタイムプロジェクションチェンバー\n\n、T.S。 Nonnenmacher 2、*、E。Atkin2、G.J。バーカー3、A。バシャリーナ-フレッシュビル1、\n\nS.B.ジョーンズ1、*\nC. Betancourt 4、S.B。 Boyd 3、D。Brailsford5、Z。Chen-Wishart6、L。Cremonesi1、A。Deisting6、A。Dias\n6、P。Dunne2、J。Haigh3、P。Hamacher-Baumann7、A。Kaboth6、A。Korzenev8、W。Ma7、P。Mermod8、\nM. Mironova 2,9、J。Monroe6、R。Nichol1、J。Nowak5、W。Parker6、H。Ritchie-Yates6、S。Roth7、\nR. Saakyan 1、N。Serra4、Y。Shitov2,10、J。Steinmann7、A。Tarrant6、M.A。Uchida2,11、S。Valder3、\nA.V. Waldron 2、M。Ward6,12、M.O。 Wascko 2\n\n2\n\n4\n\n7\n\n1ユニバーシティ・カレッジ・ロンドン、ガワー・ストリート、キングス・クロス、ロンドンWC1E 6BT、物理天文学科、\n\nイギリス; anastasia.freshville@ucl.ac.uk（A.B.-F。）; l.cremonesi@ucl.ac.uk（L.C。）; r.nichol@ucl.ac.uk（R.N。）;\nr.saakyan@ucl.ac.uk（R.S。）\nThe Blackett Laboratory、Imperial College London、London SW7 2BW、英国; e.atkin17@imperial.ac.uk（E.A。）;\np.dunne12@imperial.ac.uk（P.D。）; maria.mironova@physics.ox.ac.uk（M.M。）; Shitov@JINR.ru（Y.S。）;\nm.a.uchida@imperial.ac.uk（M.A.U。）; a.waldron@imperial.ac.uk（A.V.W。）; m.wascko@im

In [66]:
gf["ja_txt"][1]

'論文\nCERNT10ビームの軸外特性評価\n高い運動量陽子測定\n圧力ガスタイムプロジェクションチェンバー\n\n、T.S。 Nonnenmacher 2、*、E。Atkin2、G.J。バーカー3、A。バシャリーナ-フレッシュビル1、\n\nS.B.ジョーンズ1、*\nC. Betancourt 4、S.B。 Boyd 3、D。Brailsford5、Z。Chen-Wishart6、L。Cremonesi1、A。Deisting6、A。Dias\n6、P。Dunne2、J。Haigh3、P。Hamacher-Baumann7、A。Kaboth6、A。Korzenev8、W。Ma7、P。Mermod8、\nM. Mironova 2,9、J。Monroe6、R。Nichol1、J。Nowak5、W。Parker6、H。Ritchie-Yates6、S。Roth7、\nR. Saakyan 1、N。Serra4、Y。Shitov2,10、J。Steinmann7、A。Tarrant6、M.A。Uchida2,11、S。Valder3、\nA.V. Waldron 2、M。Ward6,12、M.O。 Wascko 2\n\n2\n\n4\n\n7\n\n1ユニバーシティ・カレッジ・ロンドン、ガワー・ストリート、キングス・クロス、ロンドンWC1E 6BT、物理天文学科、\n\nイギリス; anastasia.freshville@ucl.ac.uk（A.B.-F。）; l.cremonesi@ucl.ac.uk（L.C。）; r.nichol@ucl.ac.uk（R.N。）;\nr.saakyan@ucl.ac.uk（R.S。）\nThe Blackett Laboratory、Imperial College London、London SW7 2BW、英国; e.atkin17@imperial.ac.uk（E.A。）;\np.dunne12@imperial.ac.uk（P.D。）; maria.mironova@physics.ox.ac.uk（M.M。）; Shitov@JINR.ru（Y.S。）;\nm.a.uchida@imperial.ac.uk（M.A.U。）; a.waldron@imperial.ac.uk（A.V.W。）; m.wascko@im

In [24]:
len(df["en_txt"][0])

73436

In [34]:
def split_en_txt(str, num):
    list_en_txt = []
    for i in range(num):
        list_en_txt.append(str[i::num])
    list_en_txt = ["".join(i) for i in zip(*list_en_txt)]
    rem = len(str) % num
    if rem:
        list_en_txt.append(str[-rem:])
    return list_en_txt
test = split_en_txt(df["en_txt"][0],1000)

In [35]:
print(len(test))

74


In [27]:
def en_convert_to_ja(txt)
    ja_txt = []
    for en_txt in test:
        print(len(en_txt))
        trans_de_ja = translator.translate(en_txt, src='en', dest='ja')
        print(trans_de_ja.text)
        ja_txt.append(trans_de_ja.text)
    return ja_txt

test = map(str, ja_txt)
ja_txt = ''.join(test)

1000
論文
CERNT10ビームの軸外特性評価
高い運動量陽子測定
圧力ガスタイムプロジェクションチェンバー

、T.S。 Nonnenmacher 2、*、E。Atkin2、G.J。バーカー3、A。バシャリーナ-フレッシュビル1、

S.B.ジョーンズ1、*
C. Betancourt 4、S.B。 Boyd 3、D。Brailsford5、Z。Chen-Wishart6、L。Cremonesi1、A。Deisting6、A。Dias
6、P。Dunne2、J。Haigh3、P。Hamacher-Baumann7、A。Kaboth6、A。Korzenev8、W。Ma7、P。Mermod8、
M. Mironova 2,9、J。Monroe6、R。Nichol1、J。Nowak5、W。Parker6、H。Ritchie-Yates6、S。Roth7、
R. Saakyan 1、N。Serra4、Y。Shitov2,10、J。Steinmann7、A。Tarrant6、M.A。Uchida2,11、S。Valder3、
A.V. Waldron 2、M。Ward6,12、M.O。 Wascko 2

2

4

7

1ユニバーシティ・カレッジ・ロンドン、ガワー・ストリート、キングス・クロス、ロンドンWC1E 6BT、物理天文学科、

イギリス; anastasia.freshville@ucl.ac.uk（A.B.-F。）; l.cremonesi@ucl.ac.uk（L.C。）; r.nichol@ucl.ac.uk（R.N。）;
r.saakyan@ucl.ac.uk（R.S。）
The Blackett Laboratory、Imperial College London、London SW7 2
1000
BW、英国; e.atkin17@imperial.ac.uk（E.A。）;
p.dunne12@imperial.ac.uk（P.D。）; maria.mironova@physics.ox.ac.uk（M.M。）; Shitov@JINR.ru（Y.S。）;
m.a.uchida@imperial.ac.uk（M.A.U。）; a.waldron@imperial.ac.uk（A.V.W。）; m.wascko@imperial.ac.uk（M.O

tすべての流出は両方のDAQで一致します。アップストリームToFのトリガー条件は
S1とS3の構成要素間の一致。 S2信号は、アップストリームToFDAQによっても記録されました
しかし、トリガーでは使用されませんでした。ダウンストリームToFのDAQは、自己トリガーモードで実行されました。
流出中にゲートが開きます。 S1カウンターとS2カウンター間の一致信号も
下流のToFDAQであり、セクション4で説明されている粒子同定（PID）分析で使用されました。

2.2。調査および座標系

T10ビームライン領域が調査され、特定のコンポーネントまでの距離が測定されました
CERN Survey、Mechatronics and Measurements（SMM）グループによる0.5mmの精度。
S1、S2、S3、S4、およびTPCフレームのそれぞれの複数のポイントで、それらの位置が測定されています。

右手の座標系の軸は次のように定義されます。

xは非ビームを指します
ˆ
図に示すように、水平方向、垂直方向にˆy、ビーム方向にˆz
1000
ure5。表示します
結果は、2つの軸外角度で表されます。θは、正の角度でˆx − ˆz平面で測定されます。
+ ˆx方向で測定され、φはˆy − ˆz平面で測定され、正の角度が測定されます。
+ ˆy方向に。原点はS1にあると見なされます。

図7は、定義された座標系を使用したビームライン内のオブジェクトの角度範囲を示しています。
上記。表1は、さまざまなビームラインコンポーネントの計算された角度範囲を示しています。
S1。表2は、T10ビームライン内のさまざまなオブジェクトの中心間の距離を示しています。これらの距離
調査チームによって収集されたデータを使用して計算されました。

Instruments 2020、4、21

28の8

ビームライン内のオブジェクトの位置（S1原点）

-6

-4

-2

0

2

θ[度]

図7.T10ビームライン内のさまざまなオブジェクトの角度位置。このビューの原点は
S1の中心;ステアリングビームの真の中心は、θで+1◦、φで0◦にあります。

表1.T内のオブジェクトの角度範囲
1000
S1から測定した10ビームライン。

オブジェクト

最小φ
最大θ
最小θ
0.36◦±

さまざまな数のモデレーターブロックの3回の飛行スペクトル。

正しい飛行時間を計算するために、ケーブル配線と機器によって引き起こされるタイミング遅延が考慮されます
アカウント。同じ方法を使用して、S1とS3の間の飛行時間の測定値を修正します。
およびS2およびS4。最初に記録された時間tiはtS1またはtS2のいずれかであり、最後に記録された時間tfはtS3です。
またはtS4、それぞれ。

タイミングオフセットは、t f −tiスペクトルの最速のピークを想定してビームラインで測定されます。
モデレートされていないデータの場合、0.8 GeV / cの運動量を持つ帯電したMIPによって生成されます。必須
タイミングシフトは、これを前提として、最速のピークを予想される位置に移動するために必要なシフトです。
仮定。このシフトは、測定されたすべての飛行時間に適用されます。この修正が実行されます
tS3 −tS1の測定とtS4−tS2の測定の両方で別々に。必要なタイミング
tS4 −tS2測定のシフトは43.7nsです。 tS3 − tS1測定の場合、
1000
edタイミングシフトは
65.0ns。

モデレーターブロックなしのデータセットに対して計算された質量分布を図17に示します。
単一粒子（tS3 − tS1）に対応するS3カウンターとS1カウンター間の時間差は、
式（1）を使用した粒子の質量m。ここで、式は自然単位系です。粒子の運動量、
pは、0.8 GeV / cであると想定されます。

m2 = p2

（cid：32）（cid：18）tS3 − tS1
xS3 − xS1

（cid：19）2

（cid：33）

− 1

、

（1）

図17の陽子とパイ中間子の質量位置は、垂直の矢印で示されています。はっきりとできる
陽子と重水素に対応する明確なピークを観察します。図の挿入図はズームされたものを示しています
MIPに対応する領域。

Instruments 2020、4、21

28の16

0

0.5

1

1.5

2

2.5

3

3.5

4

4.5

m2 [GeV2 c-4]

図17.モデレーターブロックなしで取得されたデータの再構築されたマススペクトル。スペクトルは
S3とS1の時間差を使用して計算されます。垂直矢印は、粒子の

wever、3つおよび4つのモデレーターブロックで、
S4で観測された陽子の総数は減少します。初期のプロトンフラックスの増加は、MIPの場合と同様です。
散乱が増加すると、より多くの陽子が軸外のS2およびS4検出器を通過するフラックス。ザ・
その後の減少は、より厚いモデレーターでの陽子のエネルギー損失が大きいためです。順番に、これ
圧力容器内の陽子の減衰につながり、S4で観測されるイベントが少なくなります。

0ブロック-流出ごとに119±2-
1ブロック-流出ごとに181±2-
2ブロック-流出ごとに158±2-
3ブロック-流出ごとに39±1-
4ブロック-流出あたり1.3±0.2

0

-6

-5

-4

-3

-2

-1
θ[度]

0

-6

-5

-4

-3

-2

-1
θ[度]

0

図26.モデレーターブロックの数と水平方向の関数としてのS4のヒットの分布
軸外角度。 （左）最小電離粒子。 （右）陽子。凡例に示されているエラーは
こぼれあたりの粒子数の統計誤差。

図27は、陽子と陽子の比率を示しています。
1000
モデレーターブロック数の関数としてのMIP、θおよび
φ。さまざまなブロック構成のすべてについて、比率はθとφの両方でフラットです。の追加で
モデレーターブロックの場合、比率は0ブロックの場合の最高レベルの0.5から4ブロックの場合の0.002に減少します。
データ。前述のように、これは内部の低エネルギー陽子の減衰によるものと考えられています
圧力容器の壁。

0ブロック-流出ごとに236±4-
1ブロック-流出ごとに698±5-
2ブロック-1049±6 /流出
3ブロック-流出ごとに1134±9
4ブロック-流出ごとに495±1-

600

500

400

300

200

100

MIP /流出/学位

0ブロック

2ブロック

4ブロック

1ブロック

3ブロック

0ブロック

1ブロック

2ブロック

3ブロック

4ブロック

0.4

0.35

0.3

0.25

0.2

0.15

0.1

0.05

プロトン/ MIP

Instruments 2020、4、21

28の23

-6

-5

-4

-3

-2

-1.5

-1

-0.5

0

0.5

1


i：ロングベースライン実験の重要性。アン。

Nucl牧師部。科学2016、66、171–195、doi：10.1146 / annurev-nucl-102115-044720。
Acero、M.A。;アダムソン、P。; Aliaga、L。;アリオン、T。; Allakhverdian、V。; Altakarli、S。;アンフィモフ、N。;アントシュキン、A。;
Aurisano、A。;戻る、A。;バックハウス、C。ニュートリノを使用したニュートリノ振動パラメータの最初の測定
およびNOvAによる反ニュートリノ。物理学レット牧師2019、123、151803、doi：10.1103 /PhysRevLett.123.151803。
アビ、B。; Acciarri、R。; Acero、M.A。;アダモフ、G。;アダムス、D。; Adinolfi、M。;アフマド、Z。; Ahmed、J。;アリオン、T。;
Alonso Monsalve、S。; etal。深部地下ニュートリノ実験（DUNE）、遠方検出器の技術設計
レポート、ボリュームII：砂丘物理学。 arXiv 2020、arXiv：2002.03005。
阿部健一;阿部徹;相原秀樹;福田恭子;隼人、Y。;黄、K。;市川晃;池田眞;井上健一;石野秀樹;
etal。レターオブインテント：ハイパーカミオカンデ実験—検出器の設計と物理的可能性—。 arXiv
2011、arXiv：1109.3262。
Acciarri、R。; Acero、M.A。;アダモウスキー、M。;
1000
アダムス、C。;アダムソン、P。; Adhikari、S。;アフマド、Z。;オルブライト、
C.H。;アリオン、T。;アマドール、E。;アンダーソン、J。ロングベースラインニュートリノ施設（LBNF）と深部地下
ニュートリノ実験（DUNE）概念設計レポート、第4巻LBNFのDUNE検出器。
arXiv 2016、arXiv：1601.02984。
Andreopoulos、C。;ベル、A。; Bhattacharya、D。;カバンナ、F。;ドブソン、J。; Dytman、S。; Gallagher、H。;グゾウスキー、
。;ハッチャー、R。; Kehayias、P。; etal。 GENIEニュートリノモンテカルロジェネレーター。核インストラ

In [28]:
trans_de_ja.text

'e、A。; Betancourt、C。;ボイド、S。B。;\nBrailsford、D。; Chen-Wishart、Z。; Cremonesi、L。; etal。光学式の高圧タイムプロジェクションチェンバー\n読み出す。 Instruments 2020、近日公開。\n\n（cid：13）著者による2020年。ライセンシーMDPI、バーゼル、スイス。この記事はオープンアクセスです\nc\nCreative Commons Attribution（CC）の利用規約に基づいて配布された記事\nBY）ライセンス（http://creativecommons.org/licenses/by/4.0/）。'

In [29]:
def map_list(list_txt)
    test = map(str, ja_txt)
    ja_txt = ''.join(test)
    return ja_txt
print(ja_txt,len(ja_txt))


論文
CERNT10ビームの軸外特性評価
高い運動量陽子測定
圧力ガスタイムプロジェクションチェンバー

、T.S。 Nonnenmacher 2、*、E。Atkin2、G.J。バーカー3、A。バシャリーナ-フレッシュビル1、

S.B.ジョーンズ1、*
C. Betancourt 4、S.B。 Boyd 3、D。Brailsford5、Z。Chen-Wishart6、L。Cremonesi1、A。Deisting6、A。Dias
6、P。Dunne2、J。Haigh3、P。Hamacher-Baumann7、A。Kaboth6、A。Korzenev8、W。Ma7、P。Mermod8、
M. Mironova 2,9、J。Monroe6、R。Nichol1、J。Nowak5、W。Parker6、H。Ritchie-Yates6、S。Roth7、
R. Saakyan 1、N。Serra4、Y。Shitov2,10、J。Steinmann7、A。Tarrant6、M.A。Uchida2,11、S。Valder3、
A.V. Waldron 2、M。Ward6,12、M.O。 Wascko 2

2

4

7

1ユニバーシティ・カレッジ・ロンドン、ガワー・ストリート、キングス・クロス、ロンドンWC1E 6BT、物理天文学科、

イギリス; anastasia.freshville@ucl.ac.uk（A.B.-F。）; l.cremonesi@ucl.ac.uk（L.C。）; r.nichol@ucl.ac.uk（R.N。）;
r.saakyan@ucl.ac.uk（R.S。）
The Blackett Laboratory、Imperial College London、London SW7 2BW、英国; e.atkin17@imperial.ac.uk（E.A。）;
p.dunne12@imperial.ac.uk（P.D。）; maria.mironova@physics.ox.ac.uk（M.M。）; Shitov@JINR.ru（Y.S。）;
m.a.uchida@imperial.ac.uk（M.A.U。）; a.waldron@imperial.ac.uk（A.V.W。）; m.wascko@imperial.ac.uk（M.O.W。）

3英国、コ

In [37]:
type(ja_txt)

str

In [ ]:
from googletrans import Translator
translator = Translator()
jn = []
for txt in a:
    print(len(txt))
    trans_de_ja = translator.translate(txt, src='en', dest='ja')
    jn.appned(trans_de_ja)
    print(trans_de_ja.text)

In [60]:
value ='t systems upstream and downstream of the TPC. The\nkinetic energy of protons reaching the TPC was successfully changed from ∼0.3 GeV without moderator\nblocks to less than 0.1 GeV with four moderator blocks (40 cm path length). The ﬂux of both protons\nand MIPs off the beam axis was increased. The ratio of protons to MIPs vary as a function of the\noff-axis angle allowing for possible optimisation of the detector to select the type of required particles.\nSimulation informed by the time of ﬂight measurements show that with four moderator blocks placed in\nthe beamline, (5.6 ± 0.1) protons with energies below 0.1 GeV per spill traversed the active TPC region.\nMeasurements of the beam composition and energy are presented.\n\nKeywords: neutrino physics; time projection chamber; high pressure; CERN; beam test; time of ﬂight; off\naxis\n\n1. Introduction\n\nOne of the major goals of the global neutrino physics programme is to explore fundamental\nsymmetries of nature linked to why we live in a matter-dom'
trans_de_ja = translator.translate(value, src='en', dest='ja')
print(trans_de_ja.text)

TPCの上流と下流のシステム。の
TPCに到達する陽子の運動エネルギーは、モデレーターなしで約0.3GeVから正常に変更されました。
4つのモデレーターブロック（パス長40cm）で0.1GeV未満にブロックします。両方の陽子の流れ
ビーム軸から外れたMIPが増加しました。プロトンとMIPの比率は、
軸外角度により、検出器を最適化して必要な粒子のタイプを選択できます。
飛行測定の時間によって通知されたシミュレーションは、4つのモデレーターブロックが配置されていることを示しています
ビームライン、（5.6±0.1）陽子は、流出あたり0.1 GeV未満のエネルギーで、アクティブなTPC領域を通過しました。
ビーム組成とエネルギーの測定値が表示されます。

キーワード：ニュートリノ物理学;タイムプロジェクションチェンバー;高圧; CERN;ビームテスト;飛行時間;オフ
軸

1.はじめに

グローバルニュートリノ物理学プログラムの主要な目標の1つは、基礎を探求することです。
自然の対称性は、なぜ私たちが物質界に住んでいるのかと関連しています
